# একটি প্রাণী বিশেষজ্ঞ ব্যবস্থা বাস্তবায়ন

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) থেকে একটি উদাহরণ।

এই নমুনায়, আমরা কিছু শারীরিক বৈশিষ্ট্যের উপর ভিত্তি করে একটি প্রাণী নির্ধারণ করার জন্য একটি সহজ জ্ঞান-ভিত্তিক ব্যবস্থা বাস্তবায়ন করব। এই ব্যবস্থা নিম্নলিখিত AND-OR গাছ দ্বারা প্রতিনিধিত্ব করা যেতে পারে (এটি সম্পূর্ণ গাছের একটি অংশ, আমরা সহজেই আরও কিছু নিয়ম যোগ করতে পারি):

![](../../../../../../translated_images/bn/AND-OR-Tree.5592d2c70187f283.webp)


## আমাদের নিজস্ব বিশেষজ্ঞ সিস্টেম শেল ব্যাকওয়ার্ড ইনফারেন্স সহ

চলুন প্রোডাকশন রুলসের ভিত্তিতে জ্ঞান উপস্থাপনার জন্য একটি সহজ ভাষা সংজ্ঞায়িত করার চেষ্টা করি। আমরা নিয়মগুলি সংজ্ঞায়িত করতে কীওয়ার্ড হিসাবে পাইথন ক্লাস ব্যবহার করব। মূলত ৩ ধরণের ক্লাস থাকবে:
* `Ask` একটি প্রশ্ন প্রতিনিধিত্ব করে যা ব্যবহারকারীর কাছে জিজ্ঞাসা করতে হবে। এটি সম্ভাব্য উত্তরগুলির সেট ধারণ করে।
* `If` একটি নিয়ম প্রতিনিধিত্ব করে, এবং এটি কেবল নিয়মের বিষয়বস্তু সংরক্ষণ করার জন্য একটি সিনট্যাকটিক সুগার।
* `AND`/`OR` হল গাছের AND/OR শাখাগুলি প্রতিনিধিত্ব করার জন্য ক্লাস। তারা কেবল ভিতরের আর্গুমেন্টগুলির তালিকা সংরক্ষণ করে। কোড সহজ করার জন্য, সমস্ত কার্যকারিতা প্যারেন্ট ক্লাস `Content`-এ সংজ্ঞায়িত।


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

আমাদের সিস্টেমে, ওয়ার্কিং মেমোরি থাকবে **ফ্যাক্টস** এর তালিকা যা **অ্যাট্রিবিউট-ভ্যালু পেয়ার** হিসেবে থাকবে। নলেজবেসকে এক বড় ডিকশনারি হিসাবে সংজ্ঞায়িত করা যেতে পারে যা অ্যাকশন (নতুন ফ্যাক্ট যা ওয়ার্কিং মেমোরিতে ইনসার্ট করা উচিত) গুলিকে শর্তের সাথে ম্যাপ করে, যা AND-OR এক্সপ্রেশন আকারে প্রকাশ করা হয়। এছাড়াও, কিছু ফ্যাক্টকে `Ask` করা যেতে পারে।


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

পিছনে ইনফারেন্স (backward inference) সম্পাদন করার জন্য, আমরা `Knowledgebase` ক্লাসটি সংজ্ঞায়িত করব। এটি থাকবে:
* কর্মরত `memory` - একটি ডিকশনারি যা বৈশিষ্ট্যগুলি মানের সাথে ম্যাপ করে
* উপরের নির্ধারিত ফরম্যাটে নলেজবেসের `rules`

দুটি প্রধান পদ্ধতি হলো:
* `get` একটি বৈশিষ্ট্যের মান পাওয়ার জন্য, প্রয়োজনে ইনফারেন্স সম্পাদন করে। উদাহরণস্বরূপ, `get('color')` রং স্লটের মান পাবে (যদি প্রয়োজন হয় তাহলে জিজ্ঞাসা করবে, এবং পরে ব্যবহার করার জন্য কর্মরত মেমরিতে মান সংরক্ষণ করবে)। যদি আমরা `get('color:blue')` জিজ্ঞাসা করি, এটি একটি রং সম্পর্কে প্রশ্ন করবে, এবং তারপর রঙের উপর ভিত্তি করে `y`/`n` মান ফেরত দেবে।
* `eval` প্রকৃত ইনফারেন্স সম্পাদন করে, অর্থাৎ AND/OR গাছ অনুসরণ করে, উপ-লক্ষ্যগুলি মূল্যায়ন করে ইত্যাদি।


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

এখন আমরা আমাদের প্রাণীজগতের জ্ঞানভাণ্ডার সংজ্ঞায়িত করব এবং পরামর্শ সম্পাদন করব। লক্ষ্য করুন যে এই কলটি আপনাকে প্রশ্ন করবে। আপনি হ্যাঁ-না প্রশ্নের জন্য `y`/`n` টাইপ করে উত্তর দিতে পারেন, অথবা দীর্ঘতর বহুনির্বাচনী প্রশ্নের জন্য সংখ্যা (0..N) নির্দিষ্ট করে উত্তর দিতে পারেন।


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## ফরওয়ার্ড ইনফারেন্সের জন্য Experta ব্যবহার করা

পরবর্তী উদাহরণে, আমরা জ্ঞান উপস্থাপনার একটি লাইব্রেরি [Experta](https://github.com/nilp0inter/experta) ব্যবহার করে ফরওয়ার্ড ইনফারেন্স বাস্তবায়ন করার চেষ্টা করব। **Experta** হল পাইথনে ফরওয়ার্ড ইনফারেন্স সিস্টেম তৈরি করার জন্য একটি লাইব্রেরি, যা ঐতিহ্যবাহী পুরনো সিস্টেম [CLIPS](http://www.clipsrules.net/index.html) এর সাথে অনুরূপ হওয়ার জন্য ডিজাইন করা হয়েছে।

আমরাও আমাদের নিজেরাই ফরওয়ার্ড চেইনিং বাস্তবায়ন করতে পারতাম বড় সমস্যাই ছাড়াই, কিন্তু সরল বাস্তবায়ন সাধারণত খুব দক্ষ নয়। আরও কার্যকর রুল ম্যাচিংয়ের জন্য একটি বিশেষ অ্যালগোরিদম [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) ব্যবহৃত হয়।


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

আমরা আমাদের সিস্টেমকে একটি ক্লাস হিসেবে সংজ্ঞায়িত করব যা `KnowledgeEngine` এর সাবক্লাস। প্রতিটি নিয়ম একটি পৃথক ফাংশনের মাধ্যমে সংজ্ঞায়িত হয় যার সাথে `@Rule` অ্যানোটেশন থাকে, যা নির্ধারণ করে কখন নিয়মটি কার্যকর হবে। নিয়মের ভিতরে, আমরা `declare` ফাংশন ব্যবহার করে নতুন তথ্য যোগ করতে পারি, এবং সেই তথ্য যুক্ত করার ফলে ফরওয়ার্ড ইনফারেন্স ইঞ্জিন দ্বারা আরও কিছু নিয়ম চালানো হবে।


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

একবার আমরা একটি জ্ঞানভান্ডার সংজ্ঞায়িত করলে, আমরা আমাদের কার্যকরী মেমোরিতে কিছু প্রাথমিক তথ্য পূরণ করি, এবং তারপর তথ্যানুসন্ধান সম্পাদনের জন্য `run()` মেথডটি কল করি। আপনি ফলাফল হিসেবে দেখতে পাবেন যে নতুন অনুমেয় তথ্য কার্যকরী মেমোরিতে যুক্ত হচ্ছে, যার মধ্যে প্রাণী সম্পর্কে চূড়ান্ত তথ্যও রয়েছে (যদি আমরা সমস্ত প্রাথমিক তথ্য সঠিকভাবে সেট করি)।


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**অস্বীকৃতি**:
এই নথিটি AI অনুবাদ সেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুশীলিত হয়েছে। আমরা যথাসাধ্য সঠিকতার প্রতি যত্নশীল হলেও, স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে তা অনুগ্রহ করে লক্ষ্য করুন। মূল নথিটি তার মাতৃভাষায় সচেতন উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানুষের অনুবাদ গ্রহণ করা উচিত। এই অনুবাদের ব্যবহারের ফলে হওয়া কোনো ভুলবোঝাবুঝি বা ভুল অর্থ গ্রহণের জন্য আমরা দায়ী নই।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
